# Merging Various Datasets

We have four files currently:
1. ores_output.json -> the output from the ORES API calls giving us the quality of the articles
2. politicians_by_country_AUG.2024.csv -> the dataset containing the politicians and their countries (Provided in the assignment)
3. population_by_country_AUG.2024.csv -> the dataset containing the population of the countries (Provided in the assignment)
4. wiki_information.csv -> the dataset containing rev_ids of the wiki articles

In this notebook, I have documented, some of the steps to taken to format some of these files better for faster access in the analysis dataset.

You can ignore this notebook if you are not interested in the details and can directly use the files present in the repository to run the 4_analysis.ipynb notebook.

In [1]:
import pandas as pd 
import json

Loading the orese output file.


### Make sure to copy the csv file from the data_files folder and place it in the path of this notebook.

In [2]:
with open("ores_output.json","r") as file:
    jsonData = json.load(file)

Checking the length to confirm all records are present

In [3]:
len(jsonData['outputs'])

7103

Extracting only the rev_id and prediction of the quality of the articles columns

Take a careful look at the try catch block since it is important to handle the missing values in the dataset.

There were 4 missing values in the dataset where there was no data or the API failed. YYou can see them in the output below.

Since there is no use of these records, I have dropped them.

In [4]:
article_dict = {}
for article in jsonData['outputs']:
    try:
        current_rev_id = list(article['enwiki']['scores'].keys())[0]
        current_pred = article['enwiki']['scores'][current_rev_id]['articlequality']['score']['prediction']
        article_dict[current_rev_id] = current_pred
    except:
        print("Error at ", current_rev_id)
        print("Error in ", article)
#jsonData['outputs'][0]['enwiki']['scores']['articlequality']['score']['prediction']

Error at  1206053852
Error in  {'httpReason': 'upstream connect error or disconnect/reset before headers. reset reason: connection failure', 'httpCode': 503}
Error at  1150849598
Error in  None
Error at  1219815222
Error in  {'httpCode': 503, 'httpReason': 'upstream connect error or disconnect/reset before headers. reset reason: connection failure'}
Error at  1240718396
Error in  {'httpCode': 503, 'httpReason': 'upstream connect error or disconnect/reset before headers. reset reason: connection failure'}


To understand the error rate of the API calls, I have calculated the error rate and it is 0.0005 which is very low and 1% as prescribed in the assignment.

In [6]:
(len(jsonData['outputs']) - len(article_dict.keys()))/len(jsonData['outputs'])

0.0005631423342249754

Length of the dataset after dropping the missing values.

In [8]:
len(list(article_dict.keys()))

7099

Writing these extracted values to a new file called articles_quality.csv

In [9]:
#write article_dict to a new csv file
with open("article_quality.csv","w") as file:
    file.write("rev_id,article_quality\n")
with open("article_quality.csv","a") as file:
    for key in article_dict.keys():
        file.write("%s,%s\n"%(key,article_dict[key]))